In [1]:
%matplotlib inline

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib import gridspec
import matplotlib.patheffects as path_effects
from IPython.display import HTML

import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
import re
import math
import baltic as bt

In [2]:
xml_path = "C:/Users/iblag/Documents/flud/run01/segment4_final.xml"

lpaths = [f"C:/Users/iblag/Documents/flud/run0{run}/segment4_final.state.glm.log"
          for run in range(1, 4)] 
tree_paths = [f"C:/Users/iblag/Documents/flud/run0{run}/segment4_final.trees.txt"
          for run in range(1, 4)] 

In [3]:
tip_dates = []
for line in open(xml_path, 'r'):
    date_match = re.search(r'<date value="([0-9.]+)"', line)
    if date_match is not None:
        tip_dates.append(date_match.group(1))

mostRecent = max(tip_dates)
print(mostRecent)

predictor_description = {}
read_matrix = False
predictors = []
description = ''
read_loc = False
locations = []
counter = 1
store = ''
print('predictors found:')
design_matrix=0
for line in open(xml_path, 'r'):
    # descriptionRegex = r'<!-- predictor [0-9]+: ([A-Za-z\_\.\(\)\-0-9\, +>]+) -->'
    matrixRegex = r'\t+<parameter id="([A-Za-z0-9_.]+)" value="([0-9.\- E]+)"\s*/>'

    if '<designMatrix' in line:
        design_matrix=1
    if design_matrix:
        matrixID = re.search(matrixRegex, line)
        # valid_description = re.search(descriptionRegex, store)
        if matrixID is not None:
            predictor = matrixID.group(1)
            predictors.append(predictor)
            print(predictor)
            counter += 1

    # descriptionID = re.search(descriptionRegex, line)
    # if descriptionID is not None:
    #     description = descriptionID.group(1)

    if 'state.dataType' in line:
        read_loc = True

    state_match = re.search(r'<state code="([A-Za-z_]+)"/>', line)
    if read_loc is True and state_match is not None:
        locations.append(state_match.group(1))

    store = line

2024.1693989071039
predictors found:
state.agro_land_perc_destination_logtransform
state.agro_land_perc_origin_logtransform
state.capital_distance_km_offsetlog
state.continent
state.GDP_per_capita_destination_logtransform
state.GDP_per_capita_origin_logtransform
state.land_boarders
state.total_cattle_stock_destination_logtransform
state.total_cattle_stock_origin_logtransform
state.total_swine_stock_destination_logtransform
state.total_swine_stock_origin_logtransform
state.cattle_trade_exp_destination_offsetlog
state.cattle_trade_exp_origin_offsetlog
state.swine_trade_exp_destination_offsetlog
state.swine_trade_exp_origin_offsetlog
state.yearly_precipitation_destination_logtransform
state.yearly_precipitation_origin_logtransform
state.mean_temperatures_destination_logtransform
state.mean_temperatures_origin_logtransform


In [4]:
def compute_hpd(samples, alpha=0.05):
    """
    Compute the 100*(1-alpha)% HPD interval for a set of samples.

    Args:
        samples (array-like): A 1D array or list of samples from a posterior distribution.
        alpha (float, optional): Significance level (1 - credible interval). 
                                 Defaults to 0.05 for a 95% HPD interval.

    Returns:
        tuple: A tuple (hpd_min, hpd_max) representing the lower and upper bounds 
               of the HPD interval.

    Docstring generated with ChatGPT 4o.
    """
    samples = np.sort(samples)
    n = len(samples)
    interval_idx_inc = int(np.floor((1 - alpha) * n))
    n_intervals = n - interval_idx_inc
    interval_width = samples[interval_idx_inc:] - samples[:n_intervals]
    
    min_idx = np.argmin(interval_width)
    hpd_min = samples[min_idx]
    hpd_max = samples[min_idx + interval_idx_inc]
    
    return hpd_min, hpd_max


burnin = 10000000


In [5]:
# Initialize the migration matrix
migration_matrix = {country: {dest_country: [] for dest_country in locations} for country in locations}

# # Print the migration matrix to verify
# print(migration_matrix)


trait= 'state'
for tree_file in tree_paths:
    burned = 0
    for line in open(tree_file, 'r'):
        tree_match = re.search(r'^tree STATE_(\d+).*', line)
        if tree_match is not None:
            state_number = int(tree_match.group(1))
            if state_number >= burnin:
                burned=1
            if burned:
                treestring_match = re.search(r'= \[.*?\] (.*);', line)
                if treestring_match:
                    treestring = treestring_match.group(1)
                    treestring= treestring+";"
                    ll = bt.make_tree(treestring)
                    tree_migration_count = {country: {dest_country: 0 for dest_country in locations} 
                                            for country in locations}
                    for k in ll.Objects:
                        cur_state = k.traits[trait]
                        if not k.parent.traits:
                            continue
                        par_state = k.parent.traits[trait]

                        if par_state != cur_state:
                            migration = '%s>%s' % (par_state, cur_state)
                            tree_migration_count[par_state][cur_state] += 1
    #update matrix with info from one tree
                        for country in tree_migration_count:
                            for dest_country in tree_migration_count[country]:
                                if country == dest_country:
                                    migration_matrix[country][dest_country] = None
                                else:
                                    migration_matrix[country][dest_country].append(tree_migration_count[country][dest_country])



In [6]:
import arviz as az

mean_migration = {country: {dest_country: None for dest_country in locations} for country in locations}
HPD_migration = {country: {dest_country: None for dest_country in locations} for country in locations}
median_migration = {country: {dest_country: None for dest_country in locations} for country in locations}

# Loop through migration_matrix
for key, value in migration_matrix.items():
    for key2, value2 in value.items():
        if value2 is not None and isinstance(value2, list) and len(value2) > 0:  # Check if value2 is a non-empty list
            mean = np.mean(value2)  # Calculate the mean of the list
            median = np.median(value2)  # Calculate the median of the list
            # Convert the list to a numpy array
            data_array = np.array(value2)
            # Calculate the 95% HPD interval
            hpd_interval = az.hdi(data_array, credible_interval=0.95)
            mean_migration[key][key2] = mean
            median_migration[key][key2] = median
            HPD_migration[key][key2] = hpd_interval  # Fixed typo: changed hdp_interval to hpd_interval


In [7]:
# Function to filter the nested dictionary
def filter_hpd(migration_data):
    """
    Filters migration data to exclude entries with None or zero-valued HPD intervals.

    This function removes destination countries with:
        - `None` values
        - NumPy arrays equal to [0, 0], which typically represent undefined or empty HPD intervals.

    Args:
        migration_data (dict): A nested dictionary where keys are origin countries and
                               values are dictionaries mapping destination countries to 
                               HPD interval values (e.g., numpy arrays or None).

    Returns:
        dict: A filtered version of the input dictionary containing only valid HPD intervals.

    Docstring generated with ChatGPT 4o.
    """
    filtered_data = {}
    
    for country, destinations in migration_data.items():
        filtered_destinations = {}
        for dest_country, value in destinations.items():
            if value is not None and not (isinstance(value, np.ndarray) and np.array_equal(value, np.array([0, 0]))):
                filtered_destinations[dest_country] = value
        if filtered_destinations:
            filtered_data[country] = filtered_destinations
    
    return filtered_data

# Get the filtered migration data
filtered_migration_data = filter_hpd(HPD_migration)

# Print the filtered results
print(filtered_migration_data)

{'Australia': {'Japan': array([0, 1])}, 'Canada': {'Italy': array([0, 1]), 'USA': array([0, 1])}, 'China': {'Japan': array([0, 1]), 'Turkey': array([0, 1])}, 'France': {'Italy': array([0, 1]), 'Turkey': array([0, 1])}, 'Italy': {'China': array([0, 1]), 'Denmark': array([0, 2]), 'France': array([0, 6]), 'Netherlands': array([0, 1]), 'Sweden': array([0, 2]), 'Turkey': array([0, 1]), 'United_Kingdom': array([0, 1])}, 'Japan': {'Australia': array([0, 2]), 'China': array([0, 1]), 'Turkey': array([0, 1]), 'USA': array([0, 1])}, 'Turkey': {'China': array([0, 1])}, 'USA': {'Australia': array([0, 1]), 'Canada': array([0, 3]), 'China': array([0, 1]), 'Denmark': array([0, 1]), 'France': array([0, 1]), 'Italy': array([0, 3]), 'Japan': array([0, 1]), 'Mexico': array([0, 2]), 'Namibia': array([0, 1])}}


In [10]:
ordered_locations=['Namibia', 'Australia', 'Japan', 'China', 'Turkey', 'Italy', 'Sweden', 'Denmark', 'France', 'Netherlands', 'United_Kingdom', 'Mexico', 'Canada', 'USA']
empty = np.full((len(ordered_locations), len(ordered_locations)), np.nan)

# Create a DataFrame to hold analysis results
mean_pd = pd.DataFrame(empty, index=ordered_locations, columns=ordered_locations)

# Populate the matrix with the posterior number of jumps
for i in ordered_locations:
    for j in ordered_locations:
        if i != j:
            mean_pd.loc[i, j] = mean_migration[i][j]

print(mean_pd)

                 Namibia  Australia     Japan     China    Turkey     Italy  \
Namibia              NaN   0.000688  0.001045  0.016876  0.000199  0.000034   
Australia       0.000371        NaN  0.089121  0.034915  0.025953  0.000070   
Japan           0.000625   0.756833       NaN  0.309769  0.236175  0.000178   
China           0.052545   0.059904  0.070282       NaN  0.205561  0.000488   
Turkey          0.000145   0.029479  0.037178  0.102855       NaN  0.001445   
Italy           0.000335   0.000743  0.001038  0.484937  0.371873       NaN   
Sweden          0.000137   0.000958  0.001358  0.000308  0.000205  0.000526   
Denmark         0.000258   0.000919  0.001516  0.000685  0.001601  0.003646   
France          0.000358   0.000841  0.001162  0.001043  0.109055  0.077455   
Netherlands     0.000273   0.001099  0.001658  0.000622  0.000329  0.000261   
United_Kingdom  0.000335   0.000952  0.001843  0.000779  0.000356  0.000756   
Mexico          0.003573   0.010034  0.014269  0.006

In [11]:
# --- Compute total branch lengths across all trees after burn-in ---
total_branch_lengths = []

for tree_file in tree_paths:
    burned = 0
    for line in open(tree_file, 'r'):
        tree_match = re.search(r'^tree STATE_(\d+).*', line)
        if tree_match is not None:
            state_number = int(tree_match.group(1))
            if state_number >= burnin:
                burned = 1
            if burned:
                treestring_match = re.search(r'= \[.*?\] (.*);', line)
                if treestring_match:
                    treestring = treestring_match.group(1) + ";"
                    ll = bt.make_tree(treestring)
                    # Add total branch length (Baltic tree stores it in .treeStats or compute manually)
                    total_length = sum(k.length for k in ll.Objects if hasattr(k, 'length'))
                    total_branch_lengths.append(total_length)

# Calculate mean total branch length across posterior trees
mean_total_branch_length = np.mean(total_branch_lengths)
print(f"Mean total branch length: {mean_total_branch_length:.3f}")

# --- Compute transition rates ---
transition_rate = pd.DataFrame(empty, index=ordered_locations, columns=ordered_locations)

for i in ordered_locations:
    for j in ordered_locations:
        if i != j:
            mean_mig = mean_migration[i][j]
            if mean_mig is not None and mean_total_branch_length > 0:
                transition_rate.loc[i, j] = mean_mig / mean_total_branch_length

print("\nTransition rate matrix (transitions per unit time):")
print(transition_rate)


Mean total branch length: 463.363

Transition rate matrix (transitions per unit time):
                     Namibia  Australia     Japan         China        Turkey  \
Namibia                  NaN   0.000001  0.000002  3.642178e-05  4.304278e-07   
Australia       8.009141e-07        NaN  0.000192  7.535128e-05  5.601082e-05   
Japan           1.348685e-06   0.001633       NaN  6.685247e-04  5.096975e-04   
China           1.133998e-04   0.000129  0.000152           NaN  4.436285e-04   
Turkey          3.129153e-07   0.000064  0.000080  2.219749e-04           NaN   
Italy           7.221773e-07   0.000002  0.000002  1.046560e-03  8.025525e-04   
Sweden          2.963213e-07   0.000002  0.000003  6.656222e-07  4.422807e-07   
Denmark         5.567453e-07   0.000002  0.000003  1.479066e-06  3.455106e-06   
France          7.729752e-07   0.000002  0.000003  2.250179e-06  2.353555e-04   
Netherlands     5.882401e-07   0.000002  0.000004  1.343267e-06  7.098165e-07   
United_Kingdom  7.2217

In [12]:
transition_rate.to_csv('transition_rate.csv')